# MNIST with pytorch

In [13]:
import os
import sys
import pickle
import gzip

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [14]:
dir_path = os.path.dirname(os.path.realpath('__file__')) # absolute path of notebook
dataset_path = os.path.join(dir_path, "../data/mnist.pkl.gz")
if not os.path.exists(dataset_path):
    print('Downloading dataset with curl ...')
    if not os.path.exists(os.path.join(dir_path, '../data')):
        os.mkdir(os.path.join(dir_path, '../data'))
    url = 'http://ericjmichaud.com/downloads/mnist.pkl.gz'
    os.system('curl -L {} -o {}'.format(url, dataset_path))
print('Download failed') if not os.path.exists(dataset_path) else print('Dataset acquired')
f = gzip.open(dataset_path, 'rb')
mnist = pickle.load(f)
f.close()
print('Loaded data to variable `mnist`')

Dataset acquired
Loaded data to variable `mnist`
